In [1]:
import sys

sys.path.append("../../src")

In [2]:
import time

import joblib
import pandas as pd

# from analysis import (
# plot_confusion_matrix,
# plot_prc_auc_curve,
# plot_roc_auc_curve,
# )
from fitpredictgo import FitPredictGo, Results

In [3]:
from catboost import CatBoostClassifier

## Загрузка данных

In [4]:
X_train = pd.read_csv("../../data/baseline/X_train.csv")
X_test = pd.read_csv("../../data/baseline/X_test.csv")
y_train = pd.read_csv("../../data/baseline/y_train.csv")
y_test = pd.read_csv("../../data/baseline/y_test.csv")

## Описание исследования

В данном исследовании (эксперименте) предлагается применить алгоритм градиентного бустинга, реализованный в библиотеке CatBoost.

Планируется осуществить обучение с подбором гиперпараметров.

## Подготовка данных 

Определим признаки для обучения и целевой признак

In [5]:
original_cat_features = []

original_num_features = [
    "age",
    "systolic_bp",
    "diastolic_bp",
    "glucose_level",
    "body_temperature",
    "heart_rate",
]

new_cat_features = ["age_group", "pressure_group"]

new_num_features = []

target = "risk_level"

cat_features = original_cat_features + new_cat_features
num_features = original_num_features + new_num_features

Установим дополнительную константу для указания количества блоков для кросс-валидации.

In [6]:
cv_folds = 5

In [7]:
scoring = "f1_weighted"

In [8]:
RANDOM_STATE = 1206

Определим перечень моделей для обучения. В словарь в будущем внесем результаты обучения моделей.

In [9]:
names = {
    "CatBoostClassifier()": None,
}

In [10]:
preprocessors = {
    "CatBoostClassifier()": [None],
}

Определим инициализационные модели для кросс-валидации

In [11]:
models = {
    "CatBoostClassifier()": CatBoostClassifier(
        verbose=25,
        n_estimators=100,
        cat_features=cat_features,
        random_seed=RANDOM_STATE,
    ),
}

Определим сетку подбора гиперпараметров

In [12]:
param_grid_cat = [
    {
        "catboostclassifier__auto_class_weights": ["Balanced", "SqrtBalanced"],
        "catboostclassifier__n_estimators": [50, 100],
        "catboostclassifier__learning_rate": [0.1, 0.25, 0.15, 0.17, 0.5, 1],
        "catboostclassifier__depth": range(4, 9),
        "catboostclassifier__l2_leaf_reg": [0.1, 0.5, 1, 2, 3],
    }
]

In [13]:
param_grids = {
    "CatBoostClassifier()": param_grid_cat,
}

Выполним обучение. Для каждой модели отображается пайп-лайн для контроля.

In [14]:
%%time

res = Results()
for name in names.keys():
    print(f"{name}, CV")
    model = FitPredictGo(name, scoring)
    start = time.time()
    result = model.fit_cv(
        preprocessors[name],
        models[name],
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_cv = round(time.time() - start, 3)
    res.update(result)
    print(f"{name}, Randomized Search")
    start = time.time()
    result = model.fit_search(
        preprocessors[name],
        models[name],
        param_grids[name],
        "Randomized",
        X_train,
        y_train["risk_level"],
        cv_folds,
        True,
    )
    time_rs = round(time.time() - start, 3)
    res.update(result)
    names[name] = model
    print(
        f"--> Done. CV Time = {time_cv} секунд,"
        f" RandomizedSearch Time = {time_rs} секунд"
        f"\n\n"
    )
print("All Done")

sorted_results = res.df.sort_values(
    by="Среднее значение метрики при CV", ascending=False
)
sorted_results

CatBoostClassifier(), CV


Pipeline(steps=[('nonetype', None),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000002627FE7FD40>)])

Learning rate set to 0.5
0:	learn: 0.8316826	total: 191ms	remaining: 18.9s
25:	learn: 0.2622475	total: 1.05s	remaining: 2.98s
50:	learn: 0.1573383	total: 1.75s	remaining: 1.68s
75:	learn: 0.1084888	total: 2.41s	remaining: 763ms
99:	learn: 0.0804049	total: 3.08s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.8341633	total: 13.8ms	remaining: 1.36s
25:	learn: 0.2698206	total: 682ms	remaining: 1.94s
50:	learn: 0.1695315	total: 1.34s	remaining: 1.29s
75:	learn: 0.1113080	total: 2.06s	remaining: 651ms
99:	learn: 0.0894592	total: 2.71s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.7925170	total: 23.3ms	remaining: 2.31s
25:	learn: 0.2830024	total: 686ms	remaining: 1.95s
50:	learn: 0.1710351	total: 1.38s	remaining: 1.33s
75:	learn: 0.1152330	total: 2.08s	remaining: 658ms
99:	learn: 0.0861643	total: 2.73s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.8173231	total: 22.9ms	remaining: 2.26s
25:	learn: 0.2906247	total: 678ms	remaining: 1.93s
50:	learn: 0.1590757	total: 1.39s	remain

Pipeline(steps=[('nonetype', None),
                ('catboostclassifier',
                 <catboost.core.CatBoostClassifier object at 0x000002627FE7FD40>)])

0:	learn: 0.8111469	total: 18ms	remaining: 884ms
25:	learn: 0.2317664	total: 577ms	remaining: 533ms
49:	learn: 0.1206197	total: 1.15s	remaining: 0us
0:	learn: 0.8326085	total: 18.8ms	remaining: 919ms
25:	learn: 0.2375670	total: 600ms	remaining: 554ms
49:	learn: 0.1271916	total: 1.14s	remaining: 0us
0:	learn: 0.7815138	total: 18ms	remaining: 880ms
25:	learn: 0.2512769	total: 560ms	remaining: 517ms
49:	learn: 0.1431708	total: 1.12s	remaining: 0us
0:	learn: 0.8020585	total: 17.9ms	remaining: 876ms
25:	learn: 0.2606749	total: 572ms	remaining: 528ms
49:	learn: 0.1187622	total: 1.13s	remaining: 0us
0:	learn: 0.8591833	total: 18.4ms	remaining: 900ms
25:	learn: 0.2907171	total: 547ms	remaining: 505ms
49:	learn: 0.1545715	total: 1.12s	remaining: 0us
0:	learn: 0.9851341	total: 14.1ms	remaining: 691ms
25:	learn: 0.4393167	total: 425ms	remaining: 392ms
49:	learn: 0.2831930	total: 837ms	remaining: 0us
0:	learn: 0.9944811	total: 13.8ms	remaining: 676ms
25:	learn: 0.5011808	total: 423ms	remaining: 39

,Наименование модели,"Время обучения, сек.","Время предсказания, сек.",Среднее значение метрики при CV,СКО метрики при CV
0,CatBoostClassifier(),2.838,0.004,0.756,0.041
1,Randomized enhanced CatBoostClassifier(),1.488,0.004,0.755,0.015


In [15]:
best_model_1 = names["CatBoostClassifier()"].get_best_estimator("Randomized")

In [16]:
print("Лучшая модель и её параметры:\n\n", best_model_1.steps[1][1].get_params())

Лучшая модель и её параметры:

 {'random_seed': 1206, 'verbose': 25, 'n_estimators': 50, 'cat_features': ['age_group', 'pressure_group'], 'learning_rate': 0.1, 'l2_leaf_reg': 2, 'depth': 7, 'auto_class_weights': 'SqrtBalanced'}


## Сохранение модели и данных для дальнейшей работы

Модель, показавшая лучший результат, чем бейзлайн модель после подбора гиперпараметров может быть сохранена, как финальная модель текущего исследования.

In [17]:
joblib.dump(best_model_1, "../../models/exp3_model_756.joblib")

['../../models/exp3_model_756.joblib']

Данные, на которых обучена лучшая модель текущего исследования не изменились. Сохранять их не будем.